# CLASIFIACION

In [32]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor
import traceback



import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from keras import layers, models
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

#from sklearn.preprocessing import LabelEncoder
#from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
# Abre el archivo JSON y lee su contenido
#with open('datos.json', 'r') as f:
#    datos_json = json.load(f)

In [3]:
import json
import os

# Lista para almacenar los datos de todos los archivos
data = []

try:
    # Obtener la lista de archivos en la carpeta preprocessed
    carpeta_preprocessed = '../Data/Preprocessed/'
    archivos_preprocessed = os.listdir(carpeta_preprocessed)
    
    # Iterar sobre cada archivo
    for archivo in archivos_preprocessed:
        # Construir la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta_preprocessed, archivo)
        
        # Leer el contenido del archivo JSON
        with open(ruta_archivo, 'r') as f:
            datos_archivo = json.load(f)
        
        # Agregar los datos del archivo a la lista de datos totales
        data.append(datos_archivo)
        
        print(f"Datos cargados exitosamente desde '{ruta_archivo}'.")
    
    print("/nDatos cargados exitosamente desde la carpeta preprocessed.")
    
except Exception as e:
    print(f"Error al cargar los datos desde {ruta_archivo}.")
    traceback.print_exc()


Datos cargados exitosamente desde '../Data/Preprocessed/S001R003.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R004.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R005.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R006.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R007.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R008.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R009.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R010.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R011.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R012.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R013.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S001R014.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S002R003.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S002R004

Datos cargados exitosamente desde '../Data/Preprocessed/S010R009.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S010R010.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S010R011.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S010R012.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S010R013.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S010R014.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R003.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R004.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R005.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R006.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R007.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R008.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R009.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S011R010

Datos cargados exitosamente desde '../Data/Preprocessed/S020R003.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R004.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R005.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R006.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R007.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R008.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R009.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R010.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R011.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R012.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R013.json'.
Datos cargados exitosamente desde '../Data/Preprocessed/S020R014.json'.
/nDatos cargados exitosamente desde la carpeta preprocessed.


In [4]:
print(len(data))
print(len(data[0]))
type(data[0][0][0])

240
64


float

## Preparación de datos

Para preparar nuestros datos, primero debemos estructurarlos en secuencias que representen las diversas actividades realizadas durante las sesiones de EEG. Cada paciente sigue un protocolo predefinido, ejecutando una serie de tareas en un orden específico que abarca desde la línea de base hasta acciones motoras e imaginadas. Este orden se presenta de la siguiente manera:

1. Baseline, eyes open
2. Baseline, eyes closed
3. Task 1 (open and close left or right fist)
4. Task 2 (imagine opening and closing left or right fist)
5. Task 3 (open and close both fists or both feet)
6. Task 4 (imagine opening and closing both fists or both feet)
7. Task 1
8. Task 2
9. Task 3
10. Task 4
11. Task 1
12. Task 2
13. Task 3
14. Task 4

In [16]:
def generar_secuencia(x):
    secuencia = [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]
    pacientes = x// 12
    res = []
    for e in range(pacientes):
        res.extend(secuencia)
    return res

In [17]:
data_clasificacion = generar_secuencia(len(data))
print(data_clasificacion)
print(f"Total registros: {len(data_clasificacion)}")
print(f"Total registros: {len(data)}")
print(f"Total de registros por canal: {len(data[0][0])}")

[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]
Total registros: 240
Total registros: 240
Total de registros por canal: 20000


Ejemplo de datos para prueba:

In [18]:
eeg_data_prueba = [
    [[47, 69, 12, 85, 52],[38, 19, 94, 25, 10]],
    [[85, 60, 17, 89, 28],[18, 17, 77, 83, 23]],
    [[79, 5, 38, 97, 26],[7, 53, 70, 82, 44]],
    [[73, 86, 99, 54, 7],[66, 42, 65, 63, 56]],
    [[20, 92, 78, 82, 63],[97, 8, 51, 13, 19]],
    [[51, 99, 24, 67, 97],[13, 40, 4, 36, 41]],
    [[28, 62, 57, 54, 100],[31, 81, 52, 46, 31]],
    [[26, 65, 16, 78, 51],[65, 44, 41, 41, 70]],
    [[97, 77, 93, 30, 61],[23, 85, 79, 60, 44]],
    [[90, 28, 14, 15, 99],[97, 26, 30, 53, 92]],
]
actividades_prueba = [1, 2, 3, 4, 5, 6, 4, 2, 2, 5]


In [19]:
datitos = np.array(data[0], dtype=np.int16)
datitos

array([[ -57,  -49,  -55, ...,    0,    0,    0],
       [ -13,  -11,  -17, ...,    0,    0,    0],
       [ -15,  -10,  -16, ...,    0,    0,    0],
       ...,
       [ -56,  -70,  -77, ...,    0,    0,    0],
       [-124, -149, -153, ...,    0,    0,    0],
       [ -28,  -40,  -37, ...,    0,    0,    0]], dtype=int16)

In [23]:
# Encontrar el máximo número de valores en un dato
max_len = max(len(registro) for sublist in data for registro in sublist)
max_len

20000

In [38]:
# Convertir a numpy arrays
#eeg_data = np.array(data).astype(np.float32)
eeg_data = np.array(data, dtype=np.float32)
actividades = np.array(data_clasificacion)

In [21]:
#CAMBIAR DATOS AQUI---------
# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(eeg_data, actividades, test_size=0.2, random_state=42)


## Creamos el modelo LSTM

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_len, len(eeg_data[0][0]))),  # Input shape: número variable de registros por electrodo
    tf.keras.layers.LSTM(64),  # Capa LSTM para aprender secuencias temporales
    tf.keras.layers.Dense(32, activation='relu'),  # Capa densa
    tf.keras.layers.Dense(6, activation='softmax')  # Capa de salida con activación softmax para clasificación
])

# Compila el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Entrenamos el modelo, añadimos verbose, para que nos muestre el proceso.

In [25]:
# Entrena el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 162s 2s/step - accuracy: 0.1552 - loss: 1.9271 - val_accuracy: 0.3333 - val_loss: 1.6883
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.3993 - loss: 1.6420 - val_accuracy: 0.3542 - val_loss: 1.7214
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.4411 - loss: 1.5265 - val_accuracy: 0.2292 - val_loss: 1.7477
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5373 - loss: 1.4594 - val_accuracy: 0.1667 - val_loss: 1.7892
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6055 - loss: 1.3371 - val_accuracy: 0.1875 - val_loss: 1.7714
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 997ms/step - accuracy: 0.5481 - loss: 1.3303 - val_accuracy: 0.3125 - val_loss: 1.7039
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 994ms/step - accuracy: 0.6004 - loss: 1.2799 - val_accuracy: 0.2083 - val_loss: 1.7148
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6897 - loss: 1.1794 - val_accuracy: 0.2708 - val_loss: 1.6384
Epoch 9/

In [26]:
# Evalúa el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data:", test_accuracy)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - accuracy: 0.2500 - loss: 1.6257
Accuracy on test data: 0.25


## Creamos el modelo CNN

Esta red neuronal convolucional (CNN) está diseñada para procesar datos de electroencefalografía (EEG) representados en un array de tres niveles. Aquí está cómo procesa los datos:

Capa de entrada (Conv1D):
La primera capa Conv1D se utiliza para convolucionar los datos de entrada.
Se especifica 64 como el número de filtros.
El tamaño del kernel es 3.
La función de activación relu se utiliza para introducir no linealidades.
La forma de entrada se especifica como (len(eeg_data[0]), len(eeg_data[0][0])), lo que corresponde al tamaño del segundo y tercer nivel del array de datos.
Capa de agrupación (MaxPooling1D):
Después de la convolución, se aplica una capa de agrupación máxima (MaxPooling1D) con un tamaño de agrupación de 2.
Esto reduce la dimensionalidad de los datos y extrae las características más importantes.
Segunda capa de convolución (Conv1D):
Se agrega otra capa Conv1D para extraer características más complejas.
Esta capa también tiene 64 filtros y un tamaño de kernel de 3.
Segunda capa de agrupación (MaxPooling1D):
Se aplica otra capa de agrupación máxima (MaxPooling1D) con un tamaño de agrupación de 2.
Capa de aplanado (Flatten):
Después de las capas de convolución y agrupación, los datos se aplanan en un vector unidimensional para alimentar las capas densas posteriores.
Capas densas (Dense):
Se agregan dos capas densas (Dense) para realizar la clasificación final.
La primera capa densa tiene 64 neuronas con una función de activación relu.
La segunda capa densa tiene 6 neuronas con una función de activación softmax.
La capa final produce una distribución de probabilidad sobre las clases, ya que se utiliza la función de activación softmax.
En cuanto al entrenamiento y evaluación del modelo:

El modelo se compila con el optimizador 'adam' y la pérdida se calcula utilizando la entropía cruzada categórica escasa ('sparse_categorical_crossentropy').
Se entrena el modelo usando los datos de entrenamiento (X_train y y_train) durante 10 épocas, validando en los datos de prueba (X_test y y_test).
Finalmente, se evalúa el modelo en los datos de prueba y se imprime la precisión obtenida.
Ten en cuenta que en el código proporcionado, deberías reemplazar X_train, y_train, X_test y y_test con los datos de entrenamiento y prueba reales.

### Modelo 1, basico

In [27]:
# Construir el modelo CNN
model_CNN = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=(len(eeg_data[0]), len(eeg_data[0][0]))),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

# Compilar el modelo
model_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_CNN.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

# Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data:", test_accuracy)

C:\Users\Fernando\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 327ms/step - accuracy: 0.2987 - loss: 195.1993 - val_accuracy: 0.2292 - val_loss: 206.5858
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step - accuracy: 0.4349 - loss: 105.2990 - val_accuracy: 0.2292 - val_loss: 130.3690
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - accuracy: 0.6605 - loss: 29.2862 - val_accuracy: 0.1667 - val_loss: 204.5600
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 287ms/step - accuracy: 0.8153 - loss: 10.6907 - val_accuracy: 0.1458 - val_loss: 208.9408
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 295ms/step - accuracy: 0.8743 - loss: 4.3895 - val_accuracy: 0.2083 - val_loss: 144.9064
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 287ms/step - accuracy: 0.9179 - loss: 4.3706 - val_accuracy: 0.1667 - val_loss: 142.0141
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 293ms/step - accuracy: 0.9530 - loss: 0.6700 - val_accuracy: 0.1875 - val_loss: 206.3477
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - accuracy: 0.9499 - loss: 6.4514 - val_accuracy

### Modelo 2, mejorar modelo basico

Este modelo es una Convolutional Neural Network (CNN) diseñada para procesar señales EEG (Electroencefalografía) para clasificarlas en diferentes categorías. Veamos cada parte detalladamente:

**Capas Convolucionales**:
La primera capa convolucional tiene 32 filtros de tamaño 3 y función de activación ReLU (Rectified Linear Unit). La forma de entrada se especifica en input_shape.
Después de cada capa convolucional, hay una capa de Max Pooling con tamaño de pool 2, lo que reduce la dimensionalidad espacial de la representación.
Este patrón se repite dos veces más, aumentando el número de filtros a 64 en la segunda capa y a 128 en la tercera capa.

**Aplanamiento (Flattening)**:
Después de las capas convolucionales y de pooling, los datos se aplanan en un vector unidimensional para poder alimentarlos a una capa densa.

**Capa Densa (Fully Connected Layer)**:
Se agrega una capa densa con 128 neuronas y función de activación ReLU.
También se aplica Dropout con una tasa del 50% para reducir el sobreajuste. El dropout elimina aleatoriamente conexiones entre neuronas durante el entrenamiento, lo que ayuda a prevenir que ciertas neuronas se vuelvan dominantes.

**Capa de Salida**:
La última capa es una capa densa con un número de neuronas igual al número de clases en el problema (en este caso, num_classes), con una función de activación softmax. Softmax convierte las puntuaciones de cada clase en probabilidades, lo que permite interpretar la salida como la probabilidad de que una entrada pertenezca a cada clase.
Compilación del Modelo:
El modelo se compila usando el optimizador Adam, que es una variante del descenso de gradiente estocástico muy efectiva para problemas de optimización.
La función de pérdida se establece como sparse_categorical_crossentropy, que es adecuada para problemas de clasificación con múltiples clases.
La métrica para evaluar el rendimiento del modelo se establece en 'accuracy' para ver cuántas clasificaciones son correctas en 
relación con el total.

**Resumen del Modelo**:
Finalmente, se muestra un resumen del modelo que detalla la arquitectura de las capas, el número de parámetros entrenables y la forma de salida de cada capa. Esto brinda una visión general de cómo está construido el modelo y cuántos parámetros tiene en total.

In [34]:
def create_eeg_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    
    # Primera capa convolucional
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    # Segunda capa convolucional
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    # Tercera capa convolucional
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    # Aplanar los datos para la capa densa
    model.add(layers.Flatten())
    
    # Capa densa con dropout para evitar el sobreajuste
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    
    # Capa de salida
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Especifica la forma de entrada y el número de clases
input_shape = (len(eeg_data[0]), len(eeg_data[0][0]))  # Por ejemplo, (longitud, número de canales)
num_classes = 4  # Por ejemplo, 2 para clasificación binaria

# Crea el modelo
model = create_eeg_cnn_model(input_shape, num_classes)

# Compila el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Muestra un resumen del modelo
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)                   │ (None, 62, 32)              │       1,920,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ (None, 31, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 29, 64)              │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ (None, 14, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_12 (Conv1D)                   │ (None, 12, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_12 (MaxPooling1D)      │ (None, 6, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,049,892 (7.82 MB)

 Trainable params: 2,049,892 (7.82 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(eeg_data, actividades, test_size=0.2, random_state=42)

# Preprocesamiento opcional (por ejemplo, normalización)
# Aquí podrías aplicar cualquier tipo de preprocesamiento necesario, como normalización de los datos de EEG

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de validación
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Loss en el conjunto de validación: {loss}')
print(f'Precisión en el conjunto de validación: {accuracy}')


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - accuracy: 0.9341 - loss: 6.6679 - val_accuracy: 0.2500 - val_loss: 117.6032
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.9200 - loss: 2.6564 - val_accuracy: 0.2292 - val_loss: 119.6634
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.8939 - loss: 2.9461 - val_accuracy: 0.2500 - val_loss: 100.2752
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - accuracy: 0.9062 - loss: 8.1336 - val_accuracy: 0.3125 - val_loss: 81.6131
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 0.9217 - loss: 2.4520 - val_accuracy: 0.2917 - val_loss: 75.0444
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.9518 - loss: 0.7998 - val_accuracy: 0.2708 - val_loss: 75.3077
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.9061 - loss: 1.2078 - val_accuracy: 0.2500 - val_loss: 70.0495
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.9314 - loss: 1.1180 - val_accuracy: 0.2917 -

In [ ]:
# Definir la función para construir el modelo CNN
def create_model(filters, kernel_size, pool_size, dense_units, learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters, kernel_size, activation='relu', input_shape=(len(eeg_data[0]), len(eeg_data[0][0]))),
        tf.keras.layers.MaxPooling1D(pool_size),
        tf.keras.layers.Conv1D(filters, kernel_size, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
    ])

    # Compilar el modelo
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Crear el modelo KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=1)

# Definir los parámetros para la búsqueda de hiperparámetros
param_grid2 = {
    'filters': [32, 64],
    'kernel_size': [3, 5],
    'pool_size': [2, 3],
    'dense_units': [64, 128],
    'learning_rate': [0.001, 0.0001]
}

param_grid = {
    'filters': [64]
}

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
print("Start...")
grid_result = grid.fit(X_train, y_train)

# Mostrar los resultados
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Start...


In [28]:
# Construir el modelo CNN con ajuste de hiperparámetros
model = tf.keras.Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(len(eeg_data[0]), len(eeg_data[0][0]))),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Capa de dropout para regularización
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')
])

# Compilar el modelo con optimizador Adam y tasa de aprendizaje ajustada
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con más épocas para permitir una mejor convergencia
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=1)

# Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data:", test_accuracy)

Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 379ms/step - accuracy: 0.1964 - loss: 137.4720 - val_accuracy: 0.1667 - val_loss: 104.4768
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 296ms/step - accuracy: 0.3082 - loss: 159.4111 - val_accuracy: 0.2917 - val_loss: 72.3042
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 350ms/step - accuracy: 0.4561 - loss: 147.1797 - val_accuracy: 0.2917 - val_loss: 116.5918
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 296ms/step - accuracy: 0.4576 - loss: 97.7726 - val_accuracy: 0.1667 - val_loss: 161.3704
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 298ms/step - accuracy: 0.4446 - loss: 158.8485 - val_accuracy: 0.3750 - val_loss: 99.3134
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 331ms/step - accuracy: 0.4801 - loss: 88.4879 - val_accuracy: 0.2083 - val_loss: 71.3148
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 313ms/step - accuracy: 0.4634 - loss: 68.6324 - val_accuracy: 0.2083 - val_loss: 71.6484
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 310ms/step - accuracy: 0.5313 - loss: 36.2485 - val_accura

# Discusión de resultados
Si la probabilidad de adivinar al azar una clasificación correcta entre 6 clases es del 16.7% (0.167), entonces podríamos considerar que el modelo está funcionando mejor que simplemente adivinar al azar. Dado que el modelo tiene una precisión del 25% en los datos de prueba, está superando la predicción aleatoria.

Sin embargo, es importante tener en cuenta que una precisión del 25% aún es relativamente baja, lo que sugiere que el modelo aún tiene margen de mejora. Esto puede ser especialmente difícil con datos de EEG, que son señales caóticas y difíciles de predecir.

Para mejorar la precisión del modelo, podrías intentar ajustar hiperparámetros del modelo, como el número de capas y neuronas, el tamaño de los filtros en las capas convolucionales, la tasa de aprendizaje del optimizador, etc. También podrías explorar técnicas de preprocesamiento de datos más avanzadas y considerar la posibilidad de utilizar arquitecturas de redes neuronales más complejas o incluso técnicas de aprendizaje profundo como redes neuronales recurrentes (RNN) si la naturaleza secuencial de los datos de EEG lo justifica. Además, recuerda que la calidad y cantidad de los datos de entrenamiento también juega un papel importante en el rendimiento del modelo.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Import useful packages
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

# Hide the Configuration and Warnings
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

import random
import numpy as np
import tensorflow as tf
from Models.DatasetAPI.DataLoader import DatasetLoader
from Models.Network.CNN import CNN
from Models.Loss_Function.Loss import loss
from Models.Evaluation_Metrics.Metrics import evaluation

# Model Name
Model = 'Convolutional_Neural_Network'

# Clear all the stack and use GPU resources as much as possible
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Your Dataset Location, for example EEG-Motor-Movement-Imagery-Dataset
# The CSV file should be named as training_set.csv, training_label.csv, test_set.csv, and test_label.csv
DIR = 'DatasetAPI/EEG-Motor-Movement-Imagery-Dataset/'
SAVE = 'Saved_Files/' + Model + '/'
if not os.path.exists(SAVE):  # If the SAVE folder doesn't exist, create one
    os.mkdir(SAVE)

# Load the dataset, here it uses one-hot representation for labels
train_data, train_labels, test_data, test_labels = DatasetLoader(DIR=DIR)
train_labels = tf.one_hot(indices=train_labels, depth=4)
train_labels = tf.squeeze(train_labels).eval(session=sess)
test_labels = tf.one_hot(indices=test_labels, depth=4)
test_labels = tf.squeeze(test_labels).eval(session=sess)

# Model Hyper-parameters
num_epoch = 300   # The number of Epochs that the Model run
keep_rate = 0.75  # Keep rate of the Dropout

lr = tf.constant(1e-4, dtype=tf.float32)  # Learning rate
lr_decay_epoch = 50    # Every (50) epochs, the learning rate decays
lr_decay       = 0.50  # Learning rate Decay by (50%)

batch_size = 64
n_batch = train_data.shape[0] // batch_size

# Define Placeholders
x = tf.placeholder(tf.float32, [None, 4096])
y = tf.placeholder(tf.float32, [None, 4])
keep_prob = tf.placeholder(tf.float32)

# Load Model Network
prediction = CNN(Input=x, keep_prob=keep_prob)

# Load Loss Function
loss = loss(y=y, prediction=prediction, l2_norm=True)

# Load Optimizer
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# Load Evaluation Metrics
Global_Average_Accuracy = evaluation(y=y, prediction=prediction)

# Merge all the summaries
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(SAVE + '/train_Writer', sess.graph)
test_writer = tf.summary.FileWriter(SAVE + '/test_Writer')

# Initialize all the variables
sess.run(tf.global_variables_initializer())
for epoch in range(num_epoch + 1):
    # U can use learning rate decay or not
    # Here, we set a minimum learning rate
    # If u don't want this, u definitely can modify the following lines
    learning_rate = sess.run(lr)
    if epoch % lr_decay_epoch == 0 and epoch != 0:
        if learning_rate <= 1e-6:
            lr = lr * 1.0
            sess.run(lr)
        else:
            lr = lr * lr_decay
            sess.run(lr)

    # Randomly shuffle the training dataset and train the Model
    for batch_index in range(n_batch):
        random_batch = random.sample(range(train_data.shape[0]), batch_size)
        batch_xs = train_data[random_batch]
        batch_ys = train_labels[random_batch]
        sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_prob: keep_rate})

    # Show Accuracy and Loss on Training and Test Set
    # Here, for training set, we only show the result of first 100 samples
    # If u want to show the result on the entire training set, please modify it.
    train_accuracy, train_loss = sess.run([Global_Average_Accuracy, loss], feed_dict={x: train_data[0:100], y: train_labels[0:100], keep_prob: 1.0})
    Test_summary, test_accuracy, test_loss = sess.run([merged, Global_Average_Accuracy, loss], feed_dict={x: test_data, y: test_labels, keep_prob: 1.0})
    test_writer.add_summary(Test_summary, epoch)

    # Show the Model Capability
    print("Iter " + str(epoch) + ", Testing Accuracy: " + str(test_accuracy) + ", Training Accuracy: " + str(train_accuracy))
    print("Iter " + str(epoch) + ", Testing Loss: " + str(test_loss) + ", Training Loss: " + str(train_loss))
    print("Learning rate is ", learning_rate)
    print('\n')

    # Save the prediction and labels for testing set
    # The "labels_for_test.csv" is the same as the "test_label.csv"
    # We will use the files to draw ROC CCurve and AUC
    if epoch == num_epoch:
        output_prediction = sess.run(prediction, feed_dict={x: test_data, y: test_labels, keep_prob: 1.0})
        np.savetxt(SAVE + "prediction_for_test.csv", output_prediction, delimiter=",")
        np.savetxt(SAVE + "labels_for_test.csv", test_labels, delimiter=",")

train_writer.close()
test_writer.close()
sess.close()